In [ ]:
import bmll2 as b2
b2.get_file('modules/auxiliary_functions.py')

In [ ]:
# can do this when i have converted the notebooks to .py files
# import auxiliary_functions
import auxiliary_functions as af

import random
import math
import pandas as pd
import numpy as np
from pandas import StringDtype

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import LogFormatterSciNotation
from scipy.stats import linregress
from scipy.optimize import curve_fit
from scipy.stats import t

In [ ]:
ticker = 'GRT'

In [ ]:
b2.get_file('top_100(Volume)/stock_data.csv')
stock_data = pd.read_csv('stock_data.csv', parse_dates = ['DateTime', 'Date'])
stock_data = stock_data.rename(columns = {'Ticker' : 'RIC'})
stock_data = stock_data.sort_values(['DateTime', 'ExchangeSequenceNo'])

In [ ]:
#b2.get_file('test_data/homo_4_immediate(100).csv')
#impact_data   = pd.read_csv('homo_4_immediate(100).csv', parse_dates = ['Date', 'Start time', 'End time'])

b2.get_file('test_data/metaorder_data_homogenous_4.csv')
impact_data = pd.read_csv('metaorder_data_homogenous_4.csv', parse_dates = ['Date', 'Start time', 'End time'])

stock_AD_data = impact_data[impact_data['RIC'] == 'GRT'][['RIC', 'Date', '20 AD volatility', '20 AD volume']]
stock_AD_data = stock_AD_data.drop_duplicates(subset = ['Date']).reset_index(drop = True)

In [ ]:
N                   = 20
trader_distribution = 'power'
alpha               = 2
identifier          = f'{trader_distribution}_{N}'

In [ ]:
%%time
# power distribution and N = 4 gave a very crude but maybe passable result
impact_profile = []

for date, day_D in GFI.groupby('Date', sort = True):
        print(date)

        trades = day_D.loc[day_D['Price'] != 0]
        N = 20
        f = af.trader_participation(N = N, method = 'power', alpha = 2, f_min = 1, f_max = trades.shape[0], seed = 1)
        c = af.cumulative_probs(f)

        if trades.empty:
            continue

        output = af.orders(N = N, trades = trades, cumulative_probs = c)
        for n in range(N):
            
            trader_n_trades = trades.iloc[output[n], ]
           
            if trader_n_trades.empty:
                continue

            trader_n_metaorders = af.metaorders(trader_n_trades)
            
            if len(trader_n_metaorders) == 0:
                continue

            else:
                for i in range(len(trader_n_metaorders)):

                    metaorder_i = trader_n_metaorders[i]
                    
                    if metaorder_i.empty:
                        continue

                    if len(metaorder_i) < 10:
                        continue

                    date           = metaorder_i['Date'].iloc[0]
                    vol_20_AD      = GFI_impact[GFI_impact['Date'] == date]['20 AD volatility'].iloc[0]
                    volume_20_AD   = GFI_impact[GFI_impact['Date'] == date]['20 AD volume'].iloc[0]
                    Q              = metaorder_i['Volume'].sum()
                    volatility     = metaorder_i['Daily Volatility']
                        
                    phi    = metaorder_i['Volume'].cumsum() / Q
                    sign   = metaorder_i['Trade Sign'].iloc[0]
                    impact = sign * np.log(metaorder_i['Mid-price after(immediate)'] / metaorder_i['Mid-price before'].iloc[0])
                    scaled_impact_vol  = impact / (vol_20_AD * np.sqrt(Q / volume_20_AD))
                    scaled_impact      = impact / (vol_20_AD * np.sqrt(Q))
                    impact_profile = pd.DataFrame({'phi': phi, 'impact': impact, 'scaled impact': scaled_impact, 'scaled impact(with volume)': scaled_impact_vol,
                                                   '20 AD volume': volume_20_AD, '20 AD volatility': vol_20_AD, 'Q': Q}).reset_index(drop = 'True')

                    if not impact_profile.empty and not impact_profile.isna().all().all():

                        impact_profile.append(impact_profile)


impact_profile  = pd.concat(impact_profile, ignore_index = True)
impact_profile.to_csv(f'{ticker}_impact_profile_{identifier}.csv', index = False)
b2.put_file(f'{ticker}_impact_profile_{identifier}.csv', 'test_data')

# takes about 9 minutes to run
